# ADS-507 Team 2 Final Project

## Load data in MySQL tables from CSV files

In [1]:
import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt
import os
import re

pd.options.display.max_rows = 17

In [2]:
# Set local environment variables to hide user name & password
user_name = os.environ['MySQLUSRAC']
user_pass = os.environ['MySQLPWDAC']

# Instantiate connection
db_conn = mysql.connect(host='localhost',
                        port=int(3306),
                        user=user_name,
                        passwd=user_pass,
                        db='507_final_proj')

In [3]:
tbl_names = pd.read_sql('SHOW TABLES', db_conn)

display(tbl_names)
print(type(tbl_names))

,Tables_in_507_final_proj
0,country_map
1,emissions_gross
2,emissions_tempy
3,iso
4,iso_tempy
5,population
6,population_tempy
7,population_trans
8,temp_core1
9,temperature


<class 'pandas.core.frame.DataFrame'>


### Update `iso` table from CSV

In [4]:
'''Using cursor, loading into temp file citation:
OpenAI. (2021). ChatGPT [Computer software]. https://openai.com/'''
# Create a cursor object
cursor = db_conn.cursor()

# Wipe temp table
ist_dlt_tble_stmnt = """DELETE FROM iso_tempy"""
cursor.execute(ist_dlt_tble_stmnt)

# Load data from CSV file into a temporary table
ist_csv_load_stmnt = """
LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/FAOSTAT_data_11-24-2020.csv'
    INTO TABLE iso_tempy
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\r\n'
IGNORE 1 ROWS
(
country_code,
country,
m49_code,
iso2,
iso3,
year_start,
year_end
)
"""
cursor.execute(ist_csv_load_stmnt)

# Insert new records into main table
ism_load_stmtn = """
INSERT INTO iso
(
country_code,
country,
m49_code,
iso2,
iso3,
year_start,
year_end
)
SELECT
tp.country_code,
tp.country,
tp.m49_code,
tp.iso2,
tp.iso3,
tp.year_start,
tp.year_end
FROM iso_tempy AS tp
LEFT JOIN iso AS mn
ON tp.iso3 = mn.iso3
WHERE mn.iso3 IS NULL
"""
cursor.execute(ism_load_stmtn)

# Wipe temp table
cursor.execute(ist_dlt_tble_stmnt)

321

### Update `emissions_gross` table from CSV

In [5]:
# Wipe temp table
egt_dlt_tble_stmnt = """DELETE FROM emissions_tempy"""
cursor.execute(egt_dlt_tble_stmnt)

# Load data from CSV file into a temporary table
egt_csv_load_stmnt = """
LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/GCB2022v27_MtCO2_flat.csv'
    INTO TABLE emissions_tempy
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\r\n'
IGNORE 1 ROWS
(
country,
iso3,
record_year,
total,
coal,
oil,
gas,
cement,
flaring,
other,
per_capita
)
"""
cursor.execute(egt_csv_load_stmnt)

# Insert new records into main table
egm_load_stmtn = """
INSERT INTO emissions_gross
(
country,
iso3,
record_year,
total,
coal,
oil,
gas,
cement,
flaring,
other,
per_capita
)
SELECT
tp.country,
tp.iso3,
tp.record_year,
tp.total,
tp.coal,
tp.oil,
tp.gas,
tp.cement,
tp.flaring,
tp.other,
tp.per_capita
FROM emissions_tempy AS tp
LEFT JOIN emissions_gross AS mn
ON tp.iso3 = mn.iso3 AND tp.record_year = mn.record_year
WHERE mn.iso3 IS NULL AND mn.record_year IS NULL
"""
cursor.execute(egm_load_stmtn)

# Wipe temp table
cursor.execute(egt_dlt_tble_stmnt)

63104

### Update `population` table from CSV

In [6]:
# Wipe temp table
ppt_dlt_tble_stmnt = """DELETE FROM population_tempy"""
cursor.execute(ppt_dlt_tble_stmnt)

# Load data from CSV file into a temporary table
ppt_csv_load_stmnt = """
LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/world_population.csv'
    INTO TABLE population_tempy
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\r\n'
IGNORE 1 ROWS
(
pop_rank,
iso3,
country,
capital,
continent,
pop_2022,
pop_2020,
pop_2015,
pop_2010,
pop_2000,
pop_1990,
pop_1980,
pop_1970,
area,
density,
grow_rate,
pop_perc
)
"""
cursor.execute(ppt_csv_load_stmnt)

# Insert new records into main table
ppm_load_stmtn = """
INSERT INTO population
(
pop_rank,
iso3,
country,
capital,
continent,
pop_2022,
pop_2020,
pop_2015,
pop_2010,
pop_2000,
pop_1990,
pop_1980,
pop_1970,
area,
density,
grow_rate,
pop_perc
)
SELECT
tp.pop_rank,
tp.iso3,
tp.country,
tp.capital,
tp.continent,
tp.pop_2022,
tp.pop_2020,
tp.pop_2015,
tp.pop_2010,
tp.pop_2000,
tp.pop_1990,
tp.pop_1980,
tp.pop_1970,
tp.area,
tp.density,
tp.grow_rate,
tp.pop_perc
FROM population_tempy AS tp
LEFT JOIN population AS mn
ON tp.iso3 = mn.iso3
WHERE mn.iso3 IS NULL
"""
cursor.execute(ppm_load_stmtn)

# Wipe temp table
cursor.execute(ppt_dlt_tble_stmnt)

234

#### Transform `population` table: Melt year cols to rows

In [7]:
'''Convert table to pandas df, melt pop numbers form cols to rows citation:
OpenAI. (2021). ChatGPT [Computer software]. https://openai.com/'''
ppm_slct_all_stmnt = """SELECT * FROM population"""
ppm_slct_all_df = pd.read_sql(ppm_slct_all_stmnt, db_conn)

# Melt the subset of columns
cols_to_melt = ['pop_2022',
                'pop_2020',
                'pop_2015',
                'pop_2010',
                'pop_2000',
                'pop_1990',
                'pop_1980',
                'pop_1970']
var_names = [re.sub(r'^pop_', '', col) for col in cols_to_melt]
ppm_slct_all_df_melted = pd.melt(ppm_slct_all_df, id_vars=['pop_rank',
                                                           'iso3',
                                                           'country',
                                                           'capital',
                                                           'continent',
                                                           'area',
                                                           'density',
                                                           'grow_rate',
                                                           'pop_perc'],
                                 value_vars=cols_to_melt,
                                 var_name='year',
                                 value_name='population')
#print(ppm_slct_all_df_melted.head())

# Insert the melted data into the MySQL table
insert_query = """
INSERT INTO population_trans (pop_rank, iso3, country, capital, continent, area, density, grow_rate, pop_perc, year, population)
SELECT %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s FROM DUAL WHERE NOT EXISTS (SELECT * FROM population_trans WHERE country = %s AND year = %s)
"""
for index, row in ppm_slct_all_df_melted.iterrows():
    variable = var_names[cols_to_melt.index(row['year'])]  # get variable name based on column name
    #print(index)
    #print(row)
    #print(variable)
    values = (row['pop_rank'],
              row['iso3'],
              row['country'],
              row['capital'],
              row['continent'],
              row['area'],
              row['density'],
              row['grow_rate'],
              row['pop_perc'],
              variable,
              row['population'],
              row['country'],
              variable)
    #print(values)
    cursor.execute(insert_query, values)

### Update `temperature` table from CSV

In [8]:
# Wipe temp table
tpt_dlt_tble_stmnt = """DELETE FROM temperature_tempy"""
cursor.execute(tpt_dlt_tble_stmnt)

# Load data from CSV file into a temporary table
tpt_csv_load_stmnt = """
LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/test.csv'
    INTO TABLE temperature_tempy
FIELDS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\r\n'
(
domain_code,
domain,
area_code,
country,
element_code,
element,
month_code,
month_name,
year_code,
record_year,
unit,
temp,
flag,
flag_desc
)
"""
cursor.execute(tpt_csv_load_stmnt)

# Insert new records into main table
tpm_load_stmtn = """
INSERT INTO temperature
(
domain_code,
domain,
area_code,
country,
element_code,
element,
month_code,
month_name,
year_code,
record_year,
unit,
temp,
flag,
flag_desc
)
SELECT
tp.domain_code,
tp.domain,
tp.area_code,
tp.country,
tp.element_code,
tp.element,
tp.month_code,
tp.month_name,
tp.year_code,
tp.record_year,
tp.unit,
tp.temp,
tp.flag,
tp.flag_desc
FROM temperature_tempy AS tp
LEFT JOIN temperature AS mn
ON tp.country = mn.country AND tp.month_code = mn.month_code AND tp.year_code = mn.year_code
WHERE mn.country IS NULL AND  mn.month_code IS NULL AND mn.year_code IS NULL
"""
cursor.execute(tpm_load_stmtn)

# Wipe temp table
cursor.execute(tpt_dlt_tble_stmnt)

229925

## Update/standardize feature values in `emissions_gross`, `temperature`, and `population_trans` tables based on mapping to `iso`

### Update `emissions_gross` table

In [9]:
'''Update table col vals based on mapping to another table citation:
OpenAI. (2021). ChatGPT [Computer software]. https://openai.com/'''
egm_updt_country_stmnt = """
UPDATE emissions_gross AS t1
INNER JOIN iso AS t2
    ON t1.iso3 = t2.iso3
SET t1.country = t2.country
WHERE t1.country <> t2.country AND t1.iso3 <> ''
"""
cursor.execute(egm_updt_country_stmnt)

egm_add_cc_stmnt = """
UPDATE emissions_gross AS t1
INNER JOIN iso AS t2
    ON t1.country = t2.country
SET t1.country_code = t2.country_code
"""
cursor.execute(egm_add_cc_stmnt)

60384

### Update `population_trans` table

In [10]:
ptm_updt_country_stmnt = """
UPDATE population_trans AS t1
INNER JOIN iso AS t2
    ON t1.iso3 = t2.iso3
SET t1.country = t2.country
WHERE t1.country <> t2.country AND t1.iso3 <> ''
"""
cursor.execute(ptm_updt_country_stmnt)

ptm_add_cc_stmnt = """
UPDATE population_trans AS t1
INNER JOIN iso AS t2
    ON t1.country = t2.country
SET t1.country_code = t2.country_code
"""
cursor.execute(ptm_add_cc_stmnt)

1864

### Update `temperature` table

In [11]:
tpm_updt_country_stmnt = """
UPDATE temperature AS t1
INNER JOIN country_map AS t2
	ON t1.country = t2.country_error
INNER JOIN iso AS t3
	ON t2.country_code = t3.country_code
SET t1.country = t3.country
WHERE t1.country <> t3.country
"""
cursor.execute(tpm_updt_country_stmnt)

tpm_add_cc_stmnt = """
UPDATE temperature AS t1
INNER JOIN iso AS t2
    ON t1.country = t2.country
SET t1.country_code = t2.country_code
"""
cursor.execute(tpm_add_cc_stmnt)

229925

## Commit changes and close cursor and connection instances

In [12]:
# Commit the changes to the database
db_conn.commit()

# Close the cursor and database connection
cursor.close()
db_conn.close()